In [17]:
import pickle
import tabula
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
df=pd.read_excel(r"../PS2/Work/Statements/New/to_train_new/latest/final_training_data.xlsx") #training_data

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bharat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
df['Description']=df['Description'].apply(str)

In [19]:
dfde=df.iloc[:,0:1] #description column i.e input feature
dfcl=df.iloc[:,2:3] # classification column i.e output feature
dfcl.columns=['Classification']
dfde.columns=['Description']

In [30]:
dfde=df.iloc[:,0:1]
dfcl=df.iloc[:,2:3]
dfcl.columns=['Classification']
dfde.columns=['Description']

In [20]:
ps=PorterStemmer()
corpus=[]
for i in dfde.index:
    des=re.sub('[^a-zA-Z0-9]+',' ',dfde['Description'][i]) #removing special characters
    des=des.lower()
    des=[ps.stem(word) for word in des]
    des=''.join(des)
    des=re.sub("\s\s+" , " ", des) #removing multiple whitespaces
    corpus.append(des)
dfcl=dfcl.values.tolist()
corpusorg=list(zip(corpus,dfcl))
corpusorg=pd.DataFrame(corpusorg)

In [21]:
X=corpusorg.iloc[:,0:1].values # corpus of words
y=corpusorg.iloc[:,1].astype(str).values # corresponding o/p classification
from sklearn.feature_extraction.text import TfidfVectorizer
ng=TfidfVectorizer(analyzer='char_wb', ngram_range=(4,7),max_features=10000)
X=ng.fit_transform(corpus).toarray() # corpus model

In [22]:
X=pd.DataFrame(X)
Y=df.iloc[:,1:2] # Type of transaction ie 2nd input feature
frames = [X,Y] # Appending corpus and Type of transaction 
df2=pd.concat(frames,axis=1)
input=df2.to_numpy()

In [23]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder() # Encoding Classification output
y=le.fit_transform(y)
from sklearn.linear_model import SGDClassifier
clf=SGDClassifier(alpha=1e-4,tol=None,random_state=42,max_iter=500)
clf.fit(input,y)

SGDClassifier(max_iter=500, random_state=42, tol=None)

In [24]:
cor = "final_10000_description_corpus.pkl"  

with open(cor, 'wb') as file:  
    pickle.dump(ng, file)
    
Pkl_Filename = "final_10000_Bank_Model.pkl"  
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(clf, file)
    
label="final_10000_classification_label.pkl"
with open(label, 'wb') as file:  
    pickle.dump(le,file)